In [1]:
import pandas as pd
import pandas as pd
from unidecode import unidecode
import re
# URL to the index CSV file
url = '../index_master.csv'

# Read the CSV file into a DataFrame
index_df = pd.read_csv(url)
index_df=index_df[index_df.year>1990]
index_df=index_df[index_df.team!='TOT']
def remove_acronyms(player_name):
    if isinstance(player_name, str):  # Only process if player_name is a string
        # Define a regex pattern to remove common suffixes/acronyms
        pattern = r'\b(Jr\.|Sr\.| Jr| Sr|II|III|IV|V|VI)\b'
        # Remove accents and non-ASCII characters
        player_name = unidecode(player_name)
        # Use regex to substitute any suffix patterns
        cleaned_name = re.sub(pattern, '', player_name).strip()
        return cleaned_name
    else:
        return player_name  # If not a string (like NaN), return as-is

# Convert non-English characters in the 'player' column to their ASCII/English equivalent
index_df['player']=index_df['player'].astype(str)

index_df['player'] = index_df['player'].apply(lambda x: unidecode(str(x)))
# Display the cleaned 'player' column to verify the conversion
team_acronym_map = {
    'NJN': 'BKN',  # New Jersey Nets -> Brooklyn Nets
    'BRK':'BKN',
    'VAN': 'MEM',  # Vancouver Grizzlies -> Memphis Grizzlies
    'CHA': 'CHA',  # Old Charlotte Hornets -> Modern Charlotte Hornets
    'NOH': 'NOP',  # New Orleans Hornets -> New Orleans Pelicans
    'SEA': 'OKC',  # Seattle SuperSonics -> Oklahoma City Thunder
    'WSB': 'WAS',  # Washington Bullets -> Washington Wizards
    'CHH': 'CHA',  # Charlotte Hornets -> Modern Charlotte Hornets
    'CHO':'CHA',
    'SDC': 'LAC',  # San Diego Clippers -> LA Clippers
    'KCK': 'SAC',  # Kansas City Kings -> Sacramento Kings
    'NOK':'NOP',

    'PHO': 'PHX',
}

index_df['team'] = index_df['team'].replace(team_acronym_map)

# Save the cleaned index DataFrame if necessary

# Proceed with the merging step with the salary data, once the salary data is loaded into 'salary_df'
# Assuming 'salary_df' is already prepared from the salary scraping
# salary_df['Player'] = salary_df['Player'].apply(lambda x: unidecode(str(x)))  # Ensure salary data also has ASCII names
# merged_df = pd.merge(salary_df, index_df, left_on='Player', right_on='player', how='inner')

# Save or process the merged data further if needed
# print(merged_df.head())
def convert_salary(salary_str):
    if isinstance(salary_str, str):  # Only process if salary_str is a string
        # Remove '$' and ',' from the string, then convert to float
        salary_num = float(salary_str.replace('$', '').replace(',', ''))
        return salary_num
    else:
        return salary_str  # If not a string, return as-is (e.g., NaN)


salary=pd.read_csv('nba_salaries_raw.csv')

salary=salary[~salary.Salary.isna()]
salary=salary[~salary.Player.isna()]


index_df['player'] = index_df['player'].apply(remove_acronyms)

# Clean 'Player' names in the salary data file
salary.rename(columns={'Player':'player','Team':'team','Year':'year','Salary':'salary'},inplace=True)
salary=salary[salary.player.str.lower()!='totals']

salary['player'] = salary['player'].apply(remove_acronyms)

print(len(salary))

salary['salary'] =salary['salary'].apply(convert_salary)

salary_df=salary.merge(index_df,on=['player','year','team'],how='left')

not_found1=salary_df[salary_df.nba_id.isna()]
not_found1=not_found1[['player','year','team','salary']]
salary_df.dropna(subset='nba_id',inplace=True)
not_found1.sort_values(by='salary')

index_small=index_df[['player','year','nba_id','bref_id','url']].reset_index(drop=True)
index_small['player']=index_small['player'].str.split(' ',expand=True)[0] + ' '+index_small['player'].str.split(' ',expand=True)[1]

index_small['player'] = index_small['player'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()
index_small=index_small.dropna(subset='player')
not_found1['player'] = not_found1['player'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()

not_found2=not_found1.merge(index_small,on=['player','year'],how='left')
salary_df=pd.concat([salary_df,not_found2])
salary_df.dropna(subset='nba_id',inplace=True)
salary_df.drop_duplicates(inplace=True)

not_found2=not_found2[not_found2.nba_id.isna()]
not_found2.drop_duplicates(inplace=True)

not_found2=not_found2[['player','year','team','salary']]
print(len(salary_df))
salary_df

17740
16128


,player,salary,team,year,url,bref_id,nba_id,team_id
0,Moses Malone,2406000.0,ATL,1991,https://www.basketball-reference.com/players/m...,malonmo01,77449.0,1610612737
1,Dominique Wilkins,2065000.0,ATL,1991,https://www.basketball-reference.com/players/w...,wilkido01,1122.0,1610612737
2,Jon Koncak,1550000.0,ATL,1991,https://www.basketball-reference.com/players/k...,koncajo01,402.0,1610612737
3,Doc Rivers,895000.0,ATL,1991,https://www.basketball-reference.com/players/r...,riverdo01,470.0,1610612737
4,Rumeal Robinson,800000.0,ATL,1991,https://www.basketball-reference.com/players/r...,robinru01,716.0,1610612737
...,...,...,...,...,...,...,...,...
2716,Kevin Porter,2237691.0,LAC,2025,https://www.basketball-reference.com/players/p...,porteke02,1629645.0,NaN
2727,Jaren Jackson,25257798.0,MEM,2025,https://www.basketball-reference.com/players/j...,jacksja02,1628991.0,NaN
2731,Scotty Pippen,2087519.0,MEM,2025,https://www.basketball-reference.com/players/p...,pippesc02,1630590.0,NaN
2751,Kevin Porter,1000000.0,OKC,2025,https://www.basketball-reference.com/players/p...,porteke02,1629645.0,NaN


In [2]:
index_small=index_df[['player','year','nba_id','team','bref_id','url']].reset_index(drop=True)

index_small=index_small[~index_small.nba_id.isin(salary_df.nba_id.tolist())]

not_found2['last']= not_found2['player'].str.split(' ',expand=True)[1]+not_found2['team']

index_small['last']= index_small['player'].str.split(' ',expand=True)[1] +index_small['team']

index_small['last']+=index_small['player'].str[0:3]


not_found2['last']+=not_found2['player'].str[0:3]

index_small.dropna(subset='last',inplace=True)

index_small.drop(columns=['player'],inplace=True)

salary2=not_found2.merge(index_small,on=['last','year','team'])
salary2=salary2[~salary2.nba_id.isin(salary_df.nba_id.tolist())]

salary2.drop(columns='last',inplace=True)
#salary2.drop_duplicates(inplace=True)
salary_df=pd.concat([salary_df,salary2])
salary_df.sort_values(by=['team','year','salary'],inplace=True)
salary_df.to_csv('salary_backup.csv',index=False)

In [3]:
temp_df=salary_df[salary_df.year==2025].reset_index()


In [4]:
played=index_df.drop_duplicates(subset=['nba_id','year']).reset_index(drop=True)
played.drop(columns=['team'],inplace=True)

played=played[['year','nba_id']]

In [5]:
played

,year,nba_id
0,1991,76001.0
1,1991,51.0
2,1991,76009.0
3,1991,149.0
4,1991,76016.0
...,...,...
16248,2025,1630243.0
16249,2025,1631311.0
16250,2025,1629629.0
16251,2025,1626196.0


In [6]:
temp=salary_df[['player','year','nba_id','salary']]
temp.drop_duplicates(subset=['year','nba_id'],inplace=True)

/tmp/ipykernel_709/2583396311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop_duplicates(subset=['year','nba_id'],inplace=True)


In [7]:
salary_df

,player,salary,team,year,url,bref_id,nba_id,team_id
12,Duane Ferrell,125000.0,ATL,1991,https://www.basketball-reference.com/players/f...,ferredu01,273.0,1610612737
11,Gary Leonard,200000.0,ATL,1991,https://www.basketball-reference.com/players/l...,leonaga01,77370.0,1610612737
9,Sidney Moncrief,510000.0,ATL,1991,https://www.basketball-reference.com/players/m...,moncrsi01,77626.0,1610612737
10,Spud Webb,510000.0,ATL,1991,https://www.basketball-reference.com/players/w...,webbsp01,892.0,1610612737
8,John Battle,590000.0,ATL,1991,https://www.basketball-reference.com/players/b...,battljo01,174.0,1610612737
...,...,...,...,...,...,...,...,...
17441,Alex Sarr,11245680.0,WAS,2025,https://www.basketball-reference.com/players/s...,sarral01,1642259.0,1610612764
17440,Marvin Bagley,12500000.0,WAS,2025,https://www.basketball-reference.com/players/b...,baglema01,1628963.0,1610612764
17439,Richaun Holmes,12648321.0,WAS,2025,https://www.basketball-reference.com/players/h...,holmeri01,1626158.0,1610612764
17437,Kyle Kuzma,23522727.0,WAS,2025,https://www.basketball-reference.com/players/k...,kuzmaky01,1628398.0,1610612764


In [8]:
smerge=pd.read_csv('salary_backup.csv')
smerge.to_csv('nba_salaries_master.csv',index=False)


In [10]:
cap=pd.read_csv('../cap.csv')

capmap=dict(zip(cap['year'],cap['Salary Cap']))
capmap

{2035: 364650000.0,
 2034: 331500000.0,
 2033: 301364000.0,
 2032: 273967000.0,
 2031: 249061000.0,
 2030: 226419000.0,
 2029: 205835000.0,
 2028: 187123000.0,
 2027: 170112000.0,
 2026: 154647000.0,
 2025: 140588000.0,
 2024: 136021000.0,
 2023: 123655000.0,
 2022: 112414000.0,
 2021: 109140000.0,
 2020: 109140000.0,
 2019: 101869000.0,
 2018: 99093000.0,
 2017: 94143000.0,
 2016: 70000000.0,
 2015: 63065000.0,
 2014: 58679000.0,
 2013: 58044000.0,
 2012: 58044000.0,
 2011: 58044000.0,
 2010: 57700000.0,
 2009: 58680000.0,
 2008: 55630000.0,
 2007: 53135000.0,
 2006: 49500000.0,
 2005: 43870000.0,
 2004: 43840000.0,
 2003: 40271000.0,
 2002: 42500000.0,
 2001: 35500000.0,
 2000: 34000000.0,
 1999: 30000000.0,
 1998: 26900000.0,
 1997: 24363000.0,
 1996: 23000000.0,
 1995: 15964000.0,
 1994: 15175000.0,
 1993: 14000000.0,
 1992: 12500000.0,
 1991: 11871000.0,
 1990: 9802000.0,
 1989: 7232000.0,
 1988: 6164000.0,
 1987: 4945000.0,
 1986: 4233000.0,
 1985: 3600000.0}

In [11]:
smerge['cap']=smerge['year'].map(capmap)
smerge

,player,salary,team,year,url,bref_id,nba_id,team_id,cap
0,Duane Ferrell,125000.0,ATL,1991,https://www.basketball-reference.com/players/f...,ferredu01,273.0,1.610613e+09,11871000.0
1,Gary Leonard,200000.0,ATL,1991,https://www.basketball-reference.com/players/l...,leonaga01,77370.0,1.610613e+09,11871000.0
2,Sidney Moncrief,510000.0,ATL,1991,https://www.basketball-reference.com/players/m...,moncrsi01,77626.0,1.610613e+09,11871000.0
3,Spud Webb,510000.0,ATL,1991,https://www.basketball-reference.com/players/w...,webbsp01,892.0,1.610613e+09,11871000.0
4,John Battle,590000.0,ATL,1991,https://www.basketball-reference.com/players/b...,battljo01,174.0,1.610613e+09,11871000.0
...,...,...,...,...,...,...,...,...,...
16234,Alex Sarr,11245680.0,WAS,2025,https://www.basketball-reference.com/players/s...,sarral01,1642259.0,1.610613e+09,140588000.0
16235,Marvin Bagley,12500000.0,WAS,2025,https://www.basketball-reference.com/players/b...,baglema01,1628963.0,1.610613e+09,140588000.0
16236,Richaun Holmes,12648321.0,WAS,2025,https://www.basketball-reference.com/players/h...,holmeri01,1626158.0,1.610613e+09,140588000.0
16237,Kyle Kuzma,23522727.0,WAS,2025,https://www.basketball-reference.com/players/k...,kuzmaky01,1628398.0,1.610613e+09,140588000.0


In [12]:
smerge['cap%']=100*smerge['salary']/smerge['cap']
smerge.sort_values(by='cap%')

,player,salary,team,year,url,bref_id,nba_id,team_id,cap,cap%
13513,Chance Comanche,5849.0,POR,2023,https://www.basketball-reference.com/players/c...,comanch01,1628435.0,1.610613e+09,123655000.0,0.004730
8174,Jacob Gilyard,5849.0,MEM,2023,https://www.basketball-reference.com/players/g...,gilyaja01,1631367.0,1.610613e+09,123655000.0,0.004730
1065,RaiQuan Gray,5849.0,BKN,2023,https://www.basketball-reference.com/players/g...,grayra01,1630564.0,1.610613e+09,123655000.0,0.004730
7643,Mac McClung,5318.0,LAL,2022,https://www.basketball-reference.com/players/m...,mccluma01,1630644.0,1.610613e+09,112414000.0,0.004731
3981,Popeye Jones,2531.0,DEN,2000,https://www.basketball-reference.com/players/j...,jonespo01,461.0,1.610613e+09,34000000.0,0.007444
...,...,...,...,...,...,...,...,...,...,...
9522,Kevin Garnett,28000000.0,MIN,2004,https://www.basketball-reference.com/players/g...,garneke01,708.0,1.610613e+09,43840000.0,63.868613
10363,Patrick Ewing,20500000.0,NYK,1998,https://www.basketball-reference.com/players/e...,ewingpa01,121.0,1.610613e+09,26900000.0,76.208178
10337,Patrick Ewing,18724000.0,NYK,1996,https://www.basketball-reference.com/players/e...,ewingpa01,121.0,1.610613e+09,23000000.0,81.408696
2292,Michael Jordan,33140000.0,CHI,1998,https://www.basketball-reference.com/players/j...,jordami01,893.0,1.610613e+09,26900000.0,123.197026


In [15]:
smerge

,player,salary,team,year,url,bref_id,nba_id,team_id,cap,cap%
0,Duane Ferrell,125000.0,ATL,1991,https://www.basketball-reference.com/players/f...,ferredu01,273.0,1.610613e+09,11871000.0,1.052986
1,Gary Leonard,200000.0,ATL,1991,https://www.basketball-reference.com/players/l...,leonaga01,77370.0,1.610613e+09,11871000.0,1.684778
2,Sidney Moncrief,510000.0,ATL,1991,https://www.basketball-reference.com/players/m...,moncrsi01,77626.0,1.610613e+09,11871000.0,4.296184
3,Spud Webb,510000.0,ATL,1991,https://www.basketball-reference.com/players/w...,webbsp01,892.0,1.610613e+09,11871000.0,4.296184
4,John Battle,590000.0,ATL,1991,https://www.basketball-reference.com/players/b...,battljo01,174.0,1.610613e+09,11871000.0,4.970095
...,...,...,...,...,...,...,...,...,...,...
16234,Alex Sarr,11245680.0,WAS,2025,https://www.basketball-reference.com/players/s...,sarral01,1642259.0,1.610613e+09,140588000.0,7.999033
16235,Marvin Bagley,12500000.0,WAS,2025,https://www.basketball-reference.com/players/b...,baglema01,1628963.0,1.610613e+09,140588000.0,8.891228
16236,Richaun Holmes,12648321.0,WAS,2025,https://www.basketball-reference.com/players/h...,holmeri01,1626158.0,1.610613e+09,140588000.0,8.996729
16237,Kyle Kuzma,23522727.0,WAS,2025,https://www.basketball-reference.com/players/k...,kuzmaky01,1628398.0,1.610613e+09,140588000.0,16.731675


In [16]:
smerge.to_csv('nba_salaries.csv',index=False)